# Peanut Butter purchase data analysis

This notebook involves two datasets from the Nielsen datasets: purchase data products data. The main task is to examine the sales pattern on two selected peanut butter UPCs. Due to data confidentiality, cell outputs are not shown.     
Questions we tried to answer :    
1) What is the distribution of purchase quantity according to different promotion types?    
2) What is average spending/trip purchase per household on each product?     
3) How JIF and Skippy's sales and price interact

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from glob import glob 

In [ ]:
%pylab inline

In [ ]:
!pwd

In [ ]:
cd ~/Documents/Codes/Retail_Analytics/Panel_Data/

In [ ]:
!ls

In [ ]:
!head ./2006/Annual_Files/trips_2006.tsv

## Load and Prepare master data frame

### Select peanut butter upc list

In [ ]:
fields = ["upc","upc_ver_uc","upc_descr","product_module_code" ,"product_module_descr", "product_group_code", 
          "brand_descr", "multi", "size1_amount", "size1_units"]
products = pd.read_csv("./Master_Files/Latest/products.tsv", sep = '\t', usecols=fields)

In [ ]:
products.dtypes

In [ ]:
## select products belong to peanut butter category
peanut_butter = products.query('product_module_code==1421 & product_group_code == 506')

In [ ]:
peanut_butter.shape

From 2004 to 2014, there are toatl 2,748 products in peanut butter category

In [ ]:
unique(peanut_butter.product_module_descr) # check if query successful

In [ ]:
peanut_butter['upc_descr']= peanut_butter['upc_descr'] + " / " + peanut_butter['size1_amount'].astype(str) + " " + peanut_butter['size1_units'] + " / " + peanut_butter['multi'].astype(str) 

In [ ]:
peanut_butter.head(2)

In [ ]:
peanut_butter = peanut_butter[["upc","upc_ver_uc","upc_descr","brand_descr","multi","size1_amount"]]

In [ ]:
peanut_butter.head(2)

### Merge pruchase file and PB upc list

In [ ]:
## make a list of fine names
pl_path = glob('./2[0-1][0-1][0-9]/Annual_Files/purchases_*.tsv')
pl_path

In [ ]:
sub_path = pl_path[4:8]

In [ ]:
sub_path

In [ ]:
print("Beginning to build Purchase Dataset: ")
pl = pd.DataFrame()
for i, path in enumerate(sub_path):
    tmp = pd.read_csv(path, sep = '\t')
    pl = pl.append(tmp)
    print(pl.shape)

In [ ]:
pl.head(2)

In [ ]:
peanut_butter.head(2)

In [ ]:
purchase = pd.merge(pl, peanut_butter, how='inner', sort=False, on=["upc","upc_ver_uc"])

In [ ]:
purchase.shape

In [ ]:
purchase.head(2)

In [ ]:
purchase.to_csv("./pb_purchase_11_14.csv")

### Merge PB purchase file with trip file

In [ ]:
tp_path = glob('./2[0-1][0-1][0-9]/Annual_Files/trips_*.tsv')
tp_path = tp_path[4:8]
tp_path

In [ ]:
fields = ['trip_code_uc', 'household_code', 'purchase_date','retailer_code', 'store_code_uc']
print("Beginning to build Purchase Dataset: ")
tp = pd.DataFrame()
for i, path in enumerate(tp_path):
    tmp = pd.read_csv(path, sep = '\t',usecols=fields)
    tp = tp.append(tmp)
    print(tp.shape)

In [ ]:
tp.head(2)

In [ ]:
purchase.head(2)

In [ ]:
purchase.shape

In [ ]:
master_df = pd.merge(purchase, tp, on="trip_code_uc", how='inner',sort=False)

In [ ]:
master_df.shape

### Subset only Grocery store data 

In [ ]:
retailers = pd.read_csv("./Master_Files/Latest/retailers.tsv", sep = '\t')

In [ ]:
retailers.head(2)

In [ ]:
master_df = pd.merge(master_df, retailers, on="retailer_code", how="inner", sort=False)

In [ ]:
master_df.shape

In [ ]:
master_df.to_csv("./master_df.csv")

In [ ]:
grocery = master_df.query('channel_type=="Grocery"')

In [ ]:
grocery.shape

In [ ]:
grocery.to_csv("./grocery_df.csv",index=False)

## Multiple purchases Analysis

### Transform variables

In [ ]:
grocery = pd.read_csv("./grocery_df.csv")

In [ ]:
grocery.head(2)

#### Compute unit_price_paid

In [ ]:
grocery['unit_price_paid'] = grocery.apply(lambda x: (x['total_price_paid']-x['coupon_value'])/x['quantity'], axis=1)

In [ ]:
grocery.head(2)

#### select Skippy CRM 16.3 and JIF 18

In [ ]:
selected_df = grocery.query('upc_descr=="SKP CRM H PLS / 16.3 OZ / 1" | upc_descr=="JIF CRM H PLS / 18.0 OZ / 1"')

In [ ]:
unique(selected_df["upc_descr"])

In [ ]:
selected_df.shape

(68197,17)

In [ ]:
min(selected_df.unit_price_paid)

In [ ]:
selected_df = selected_df.query("unit_price_paid !=0")

In [ ]:
selected_df.shape

In [ ]:
min(selected_df.unit_price_paid)

In [ ]:
selected_df['unit_price_paid'].describe()

In [ ]:
def price_bin(row):
    if row['unit_price_paid']>=0 and row['unit_price_paid']<1.6 :
        return '[0,1.6)'
    if row['unit_price_paid']>=1.6 and row['unit_price_paid']<2.25 :
        return '[1.6,2.25)'
    if row['unit_price_paid']>=2.25 and row['unit_price_paid']<2.5 :
        return '[2.25,2.5)'
    if row['unit_price_paid']>=2.5 and row['unit_price_paid']<2.6 :
        return '[2.5,2.6)'
    if row['unit_price_paid']>=2.6 and row['unit_price_paid']<3 :
        return '[2.6,3)'
    return '[3, Inf) '

In [ ]:
selected_df['unit_P_range'] = selected_df.apply (lambda row: price_bin(row),axis=1)

In [ ]:
selected_df.head(2)

In [ ]:
selected_df.groupby(['brand_descr','unit_P_range','deal_flag_uc'])['trip_code_uc'].count().unstack(2)

#### Examine basic statistics

In [ ]:
# how many households involved
households = len(unique(selected_df.household_code))
households

In [ ]:
trips = len(unique(selected_df.trip_code_uc))
trips

In [ ]:
trips/households

In [ ]:

len(unique(selected_df.store_code_uc))

In [ ]:
# how many trip purchases for each brand
selected_df.groupby(['brand_descr'])['trip_code_uc'].count()

In [ ]:
sum(selected_df.groupby(['brand_descr'])['trip_code_uc'].count())

In [ ]:
JIF = selected_df.query('brand_descr=="JIF"')
JIF_gb = JIF.groupby(['household_code'])[['trip_code_uc']].count()

In [ ]:
len(unique(JIF.trip_code_uc))

In [ ]:
len(unique(JIF.household_code))

In [ ]:
len(unique(JIF.trip_code_uc))/len(unique(JIF.household_code))

In [ ]:
bin1 = numpy.linspace(1, 30, 30)

In [ ]:
plt.hist(JIF_gb.trip_code_uc, bin1)
plt.title("Number of trips involving JIF per households")
plt.xlabel("Number of trips")
plt.ylabel("household counts")

In [ ]:
SKIPPY = selected_df.query('brand_descr=="SKIPPY"')
SKIPPY_gb = SKIPPY.groupby(['household_code'])[['trip_code_uc']].count()
SKIPPY_gb.sum()

In [ ]:
len(unique(SKIPPY.trip_code_uc))

In [ ]:
len(unique(SKIPPY.household_code))

In [ ]:
len(unique(SKIPPY.trip_code_uc))/len(unique(SKIPPY.household_code))

In [ ]:
plt.hist(SKIPPY_gb.trip_code_uc, bin1)
plt.title("Number of trips involving SKIPPY per households")
plt.xlabel("Number of trips")
plt.ylabel("household counts")

#### How many households purchase JIF and Skippy

In [ ]:
brands_hh = selected_df.groupby('household_code').brand_descr.nunique()
brands_hh.value_counts()

In [ ]:
ls = brands_hh[brands_hh==2]
ls.index

In [ ]:
len(ls.index)/brands_hh.shape[0]

In [ ]:
brands_tp = selected_df.groupby('trip_code_uc').brand_descr.nunique()
brands_tp.value_counts()

In [ ]:
63/(63+65663)

#### plot histogram according to deal_flag_uc

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 6, 6

In [ ]:
bins = numpy.linspace(0, 25, 25)

In [ ]:
sub1 = selected_df.query('upc_descr=="JIF CRM H PLS / 18.0 OZ / 1" & deal_flag_uc==1')
plt.hist(sub1.quantity,bins)
plt.title("Purchase quantity for JIF CRM 18OZ if there is a deal")
plt.xlabel("purchase quantity per trip")
plt.ylabel("counts")

In [ ]:
sub2 = selected_df.query('upc_descr=="JIF CRM H PLS / 18.0 OZ / 1" & deal_flag_uc==0')
plt.hist(sub2.quantity,bins)
plt.title("Purchase quantity for JIF CRM 18OZ if there is NO deal")
plt.xlabel("purchase quantity per trip")
plt.ylabel("counts")

In [ ]:
sub3 = selected_df.query('upc_descr=="SKP CRM H PLS / 16.3 OZ / 1" & deal_flag_uc==1')
plt.hist(sub3.quantity,bins)
plt.title("Purchase quantity for SKIPPY CRM 16.3OZ if there is a deal")
plt.xlabel("purchase quantity per trip")
plt.ylabel("counts")

In [ ]:
sub4 = selected_df.query('upc_descr=="SKP CRM H PLS / 16.3 OZ / 1" & deal_flag_uc==0')
plt.hist(sub4.quantity,bins)
plt.title("Purchase quantity for SKIPPY CRM 16.3OZ if there is NO deal")
plt.xlabel("purchase quantity per trip")
plt.ylabel("counts")

In [ ]:
rcParams['figure.figsize'] = 17, 6

In [ ]:
sub6 = selected_df.query('upc_descr=="JIF CRM H PLS / 18.0 OZ / 1"')
sub6 = sub6[['quantity','unit_P_range','deal_flag_uc']]
sub6.boxplot(by=['unit_P_range','deal_flag_uc'])
plt.title("JIF - Distribution of purchase quantity per trip according to price and deal (0=no deal)")
plt.ylabel("Purchase quantity")

In [ ]:
sub5 = selected_df.query('upc_descr=="SKP CRM H PLS / 16.3 OZ / 1"')
sub5 = sub5[['quantity','unit_P_range','deal_flag_uc']]
sub5.boxplot(by=['unit_P_range','deal_flag_uc'])
plt.title("Skippy - Distribution of purchase quantity per trip according to price and deal (0=no deal)")
plt.ylabel("Purchase quantity")

In [ ]:
selected_df.query(' retailer_code==89').groupby(['upc_descr','unit_P_range','deal_flag_uc'])['trip_code_uc'].count().unstack(2)

In [ ]:
tmp=selected_df.query(' retailer_code==89')
len(unique(tmp.store_code_uc))

In [ ]:
sub7 = selected_df.query('upc_descr=="JIF CRM H PLS / 18.0 OZ / 1" & retailer_code==89')
sub7 = sub7[['quantity','unit_P_range','deal_flag_uc']]
sub7.boxplot(by=['unit_P_range','deal_flag_uc'])
plt.title("JIF - Distribution of purchase quantity per trip according to price and deal (0=no deal), retailer:89")
plt.ylabel("Purchase quantity")
plt.ylim((0,8))

In [ ]:
sub8 = selected_df.query('upc_descr=="SKP CRM H PLS / 16.3 OZ / 1" & retailer_code==89')
sub8 = sub8[['quantity','unit_P_range','deal_flag_uc']]
sub8.boxplot(by=['unit_P_range','deal_flag_uc'])
plt.title("Skippy - Distribution of purchase quantity per trip according to price and deal (0=no deal), retailer:89")
plt.ylabel("Purchase quantity")
plt.ylim((0,8))

#### How JIF and Skippy's sales and price interact

In [ ]:
tmp.head(2)

In [ ]:
# tmp.store_code_uc.value_counts()
## below stores have most trips
#2751743    61 (#trip)
#7346835    60
#742472     53

In [ ]:
tmp['purchase_date'] = pd.to_datetime(tmp['purchase_date'])

In [ ]:
tmp.dtypes

In [ ]:
rcParams['figure.figsize'] = 6, 6

In [ ]:
sub7 = selected_df.query('upc_descr=="SKP CRM H PLS / 16.3 OZ / 1" & deal_flag_uc==1')
plt.scatter(x=sub7.unit_price_paid, y=sub7.quantity)
plt.title("Scatter plot of unit_price and quantity - SKIPPY CRM 16.3OZ (with deal)")
plt.xlabel("Unit_price_paid")
plt.ylabel("Quantity")

In [ ]:
sub8 = selected_df.query('upc_descr=="SKP CRM H PLS / 16.3 OZ / 1" & deal_flag_uc==0')
plt.scatter(x=sub8.unit_price_paid, y=sub8.quantity)
plt.title("Scatter plot of unit_price and quantity - SKIPPY CRM 16.3OZ (No deal)")
plt.xlabel("Unit_price_paid")
plt.ylabel("Quantity")